# Unsupervised Image-image Translation(UNIT) TensorFlow Implementation

In [1]:
import tensorflow as tf

In [8]:
#Funciton to chose the activation function
def activate(linear, activation='leaky_relu'):
        if activation == 'sigmoid':
            return tf.nn.sigmoid(linear)
        elif activation == 'softmax':
            return tf.nn.softmax(linear)
        elif activation == 'tanh':
            return tf.nn.tanh(linear)
        elif activation == 'relu':
            return tf.nn.relu(linear)
        elif activation == 'leaky_relu':
            return tf.nn.leaky_relu(linear)
        elif activation == 'linear':
            return linear

In [11]:
def create_conv_layer(input,           # The previous layer.
                   num_input_channels, # Num. channels in prev. layer.
                   filter_size,        # Width and height of each filter.
                   num_filters,        # Number of filters.
                   use_pooling=False,
                   padding = 'SAME'
                   strides=[1,1,1,1]
                   activation='leaky_relu'): # Use 2x2 max-pooling.

    # Shape of the filter-weights for the convolution.
    # This format is determined by the TensorFlow API.
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Create new weights aka. filters with the given shape.
    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))

    # Create new biases, one for each filter.
    biases = tf.Variable(tf.constant(1, shape=[length]))

    # Create the TensorFlow operation for convolution.
    # Note the strides are set to 1 in all dimensions.
    # The first and last stride must always be 1,
    # because the first is for the image-number and
    # the last is for the input-channel.
    # But e.g. strides=[1, 2, 2, 1] would mean that the filter
    # is moved 2 pixels across the x- and y-axis of the image.
    # The padding is set to 'SAME' which means the input image
    # is padded with zeroes so the size of the output is the same.
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=strides,
                         padding=padding)

    # Add the biases to the results of the convolution.
    # A bias-value is added to each filter-channel.
    layer += biases

    # Use pooling to down-sample the image resolution?
    if use_pooling:
        # This is 2x2 max-pooling, which means that we
        # consider 2x2 windows and select the largest value
        # in each window. Then we move 2 pixels to the next window.
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    # Activation of the layers (ReLU).
    layer = activate(layer, activation='leaky_relu')

    return layer, weights

In [12]:
def batch_normalization_layer(input_layer):
    
    dimension = input_layer.get_shape().as_list()[-1]
    
    mean, variance = tf.nn.moments(input_layer, axes=[0, 1, 2])
    
    beta = tf.get_variable('beta', dimension, tf.float32,
                               initializer=tf.constant_initializer(0.0, tf.float32))
    
    gamma = tf.get_variable('gamma', dimension, tf.float32,
                                initializer=tf.constant_initializer(1.0, tf.float32))
    
    bn_layer = tf.nn.batch_normalization(input_layer, mean, variance, beta, gamma, BN_EPSILON=1e-10)

    return bn_layer

In [15]:
def create_res_block(x, num_input_channels=3, filter_size=3, num_filters=1):
    
    residual = x
    
    layer_conv1, _ = create_conv_layer(input=x,
                                       num_input_channels=num_input_channels,
                                       filter_size=filter_size,
                                       num_filters=num_filters,
                                       use_pooling=False,
                                       activation='linear')
    
    layer_bn1 = batch_normalization_layer(layer_conv1)

    layer_norm1 = activate(layer_bn1, activation='relu')
    
    layer_conv2, _ = create_conv_layer(input=layer_norm1,
                                       num_input_channels=num_input_channels,
                                       filter_size=filter_size,
                                       num_filters=num_filters,
                                       use_pooling=False,
                                       activation='linear')
    
    layer_bn2 = batch_normalization_layer(layer_conv2)
    
    layer_bn2 += residual
    
    layer_out = activate(layer_bn2, activation='relu')
    
    return layer_out

In [ ]:
def create_encoders(config, parameters):

In [ ]:
def create_generators(config, parameters):

In [ ]:
def create_discrimiators(config, parameters):